# Datagenerering til DigDirCamp 2020

av Bjørn Christian Weinbach, DigDir Campar.

## Bakgrunn

I vår applikasjon skal me demonstrere korleis tilgang til eins eigne persondata kan gjere det enklare å navigere i den norske offentlege sektor, spesielt for ungdom. Denne dataen er ikkje tilgjengeleg vha API i dag og vi er nøydd å generere denne dataen sjølv. Denne jupyter notebooken inneheld python-kode for generering av syntetiske data for ein populasjon som blei brukt som ein del av forarbeidet til prosjektet til DigDir Camparane i 2020.

## Naudsynte imports

In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
import datetime

## Globale variablar

$$\text{testpnr} = \text{Test-personnummer}$$

In [ ]:
testpnr = ['23079412414',
           '23079412333',
           '23079412252',
           '23079412171',
           '31010650596',
           '27080350274',
           '04090751315',
           '18060550537',
           '23090350056',
           '23079418366',
           '23079418285',
           '23079418013',
           '23079417815']

## Globale Funksjonar

In [ ]:
def tlfnr():
    """Return phone no in the norwegian style."""
    return random.randint(10000000, 99999999)

def randomName(data):
    """Return random norwegian male or female name."""
    if random.randint(0, 1) == 1:   
        malename = np.array(data["Herrenavn"].sample(1, replace = True))
        malelastname = np.array(data["Etternavn"].sample(1, replace = True))
        return (malename + " " + malelastname)[0], (malename + "." + malelastname + "@norge.no")[0]
    else:
        femalename = np.array(data["Kvinnenavn"].sample(1, replace = True))
        femalelastname = np.array(data["Etternavn"].sample(1, replace = True))
        return (femalename + " " + femalelastname)[0], (femalename + "." + femalelastname + "@norge.no")[0]

## Last inn datasett

### Kjelder

Norske navn: https://no.wikipedia.org/wiki/Liste_over_norske_mannsnavn , https://no.wikipedia.org/wiki/Liste_over_norske_kvinnenavn og https://no.wikipedia.org/wiki/Liste_over_norske_etternavn

Postnummer: https://data.norge.no/datasets/5e6847ba-156d-4e14-85d3-8d7f8b727523

In [ ]:
data = pd.read_csv('navn.csv')
post = pd.read_csv('postnr.csv', converters={'field_postnummer': lambda x: str(x)}, delimiter=';')

## Kombiner til ein heil populasjon

In [ ]:
titl = []
pnum = []
name = []
email = []
fnum = []
vig = []
adress = []
i = 0
for t in testpnr:
    titl.append("Node " + str(i))
    nam, ema = randomName(data)
    pnum.append(t)
    name.append(nam)
    email.append(ema)
    fnum.append(tlfnr())
    adress.append("Test adresse " + str(random.randint(1, 100)))
    i += 1

populasjon = pd.DataFrame()
populasjon['title'] = titl
populasjon['field_personnummer'] = pnum
populasjon['field_namn'] = name
populasjon['field_epost'] = email
populasjon['field_tlf'] = fnum
populasjon['field_adresse'] = adress
populasjon[post.columns] = pd.concat([post.sample(len(testpnr), replace = True, axis = 0)], axis=0, ignore_index=True)
populasjon.reset_index(drop=True)
populasjon = populasjon.set_index("title")
populasjon

## Populasjon -> CSV for Drupal import

In [ ]:
populasjon.to_csv('personer.csv')

## VIGO-> CSV for Drupal import

In [ ]:
vigo = pd.DataFrame()
vigo['title'] = titl
vigo['field_personnummer_vigo'] = pnum
vigo['field_rle_karakter'] = np.random.randint(1, 6, len(testpnr))
vigo['field_naturfag_karakter'] = np.random.randint(1, 6, len(testpnr))
vigo['field_matematikk_karakter'] = np.random.randint(1, 6, len(testpnr))
vigo['field_rle_fravaer'] = abs(np.random.normal(0.05, 0.05, len(testpnr)))
vigo['field_naturfag_fravaer'] = abs(np.random.normal(0.05, 0.05, len(testpnr)))
vigo['field_matematikk_fravaer'] = abs(np.random.normal(0.05, 0.05, len(testpnr)))
vigo = vigo.reset_index(drop=True)
vigo = vigo.set_index("title")
vigo.to_csv('vigo.csv')
vigo

## Helsenorge-fastlege -> CSV for SQL import

In [ ]:
populasjon['field_poststad'][0]

In [ ]:
docnam = []
clinic = []
i = 0
for _ in testpnr:
    tmp1, _ = randomName(data)
    tmp2 = np.array(populasjon['field_poststad'])[i] + ' Helsesenter'
    docnam.append(tmp1)
    clinic.append(tmp2)
    i += 1

health = pd.DataFrame()
health['title'] = titl
health['field_personnummer_helse'] = pnum
health['field_fastlege'] = docnam
health['field_legesenter'] = clinic
health = health.reset_index(drop=True)
healthgp = health.set_index("title")
healthgp.to_csv('helse.csv')
health